## 01_data_preprocessing.ipynb
**Project:** Stablecoin Liquidity as a Proxy for Capital Allocation  
**Purpose:** Load raw datasets (CoinGecko stablecoins, FRED macro variables, optional DeFi data), clean and merge them, and save preprocessed CSVs for analysis

In [4]:
import requests
from datetime import datetime
import time
import pandas as pd 
import numpy as np
import os
from pycoingecko import CoinGeckoAPI
from fredapi import Fred

In [5]:
os.chdir('/Users/erikasohn/Library/Mobile Documents/com~apple~CloudDocs/Documents/classes/khoury')
print("\nFiles in current directory:")
print(os.listdir('.'))


Files in current directory:
['.DS_Store', 'ds3500_fa25', 'ds3500_source', 'ds4200_fa25', 'ds3500-hw7', 'cs3200_fa25']


#### Data Gathering 

In [6]:
# --- Get FRED data --- #  
rwa_protocols = [
    'ondo-finance',
    'maple-finance', 
    'goldfinch',
    'centrifuge',
    'truefi',
    'credix',
    'ribbon-lend',
    'polytrade',
    'swarm-markets'
]

def fetch_protocol_data(protocol_slug):
    """Fetch TVL history from defilamma"""
    url = f"https://api.llama.fi/protocol/{protocol_slug}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        # Extract TVL (per contract) in time series
        records = []
        if 'tvl' in data:
            for entry in data['tvl']:
                records.append({
                    'date': datetime.fromtimestamp(entry['date']),
                    'protocol': protocol_slug,
                    'tvl': entry['totalLiquidityUSD']
                })
        
        # Extract chain TVL (per blockchain, specific)
        if 'chainTvls' in data:
            for chain, chain_data in data['chainTvls'].items():
                if isinstance(chain_data, dict) and 'tvl' in chain_data:
                    for entry in chain_data['tvl']:
                        records.append({
                            'date': datetime.fromtimestamp(entry['date']),
                            'protocol': protocol_slug,
                            'chain': chain,
                            'tvl': entry['totalLiquidityUSD']
                        })
        
        return pd.DataFrame(records)
    
    except Exception as e:
        print(f"Error fetching {protocol_slug}: {e}")
        return pd.DataFrame()

# Fetch data 
all_data = []
for protocol in rwa_protocols:
    print(f"Fetching {protocol}...")
    df = fetch_protocol_data(protocol)
    if not df.empty:
        all_data.append(df)
    time.sleep(0.5)  # Be nice to the API

# Create combined df
rwa_df = pd.concat(all_data, ignore_index=True)
print(f"\nTotal observations: {len(rwa_df)}")
print(f"Columns: {rwa_df.columns.tolist()}")
print(f"Date range: {rwa_df['date'].min()} to {rwa_df['date'].max()}")

print(f"\nShape: {rwa_df.shape}")
print(f"Date range: {rwa_df['date'].min()} to {rwa_df['date'].max()}")
print("\nFirst few rows:")
print(rwa_df.head())
print("\nLast few rows:")
print(rwa_df.tail())

Fetching ondo-finance...
Fetching maple-finance...
Fetching goldfinch...
Fetching centrifuge...
Fetching truefi...
Fetching credix...
Fetching ribbon-lend...
Fetching polytrade...
Fetching swarm-markets...

Total observations: 59973
Columns: ['date', 'protocol', 'tvl', 'chain']
Date range: 2020-10-17 20:00:00 to 2025-11-29 16:11:47

Shape: (59973, 4)
Date range: 2020-10-17 20:00:00 to 2025-11-29 16:11:47

First few rows:
                 date      protocol  tvl chain
0 2023-01-26 19:00:00  ondo-finance  135   NaN
1 2023-01-27 19:00:00  ondo-finance  315   NaN
2 2023-01-28 19:00:00  ondo-finance  315   NaN
3 2023-01-29 19:00:00  ondo-finance  315   NaN
4 2023-01-30 19:00:00  ondo-finance  495   NaN

Last few rows:
                     date       protocol   tvl    chain
59968 2025-11-25 19:00:00  swarm-markets  1589  Polygon
59969 2025-11-26 19:00:00  swarm-markets  1891  Polygon
59970 2025-11-27 19:00:00  swarm-markets  2010  Polygon
59971 2025-11-28 19:00:00  swarm-markets  2035  Polyg

In [7]:
# --- Get Stablecoin data --- #  
def get_stablecoin_history(stablecoin_id):
    """
    Get stablecoin data from DeFiLlama
    1 = USDC, 2 = USDT, 7 = DAI
    """
    url = f"https://stablecoins.llama.fi/stablecoin/{stablecoin_id}"
    response = requests.get(url)
    data = response.json()
    
    records = []
    if 'tokens' in data:
        for entry in data['tokens']:
            records.append({
                'date': datetime.fromtimestamp(entry['date']),
                'mcap': entry['circulating']['peggedUSD']
            })
    
    return pd.DataFrame(records)

# Get main stablecoins
print("Fetching USDC...")
usdc_df = get_stablecoin_history(1)
usdc_df.columns = ['date', 'usdc_mcap']

print("Fetching USDT...")
usdt_df = get_stablecoin_history(2)
usdt_df.columns = ['date', 'usdt_mcap']

print("Fetching DAI...")
dai_df = get_stablecoin_history(7)
dai_df.columns = ['date', 'dai_mcap']

# Merge stablecoins
stablecoin_df = usdc_df.merge(usdt_df, on='date', how='outer')
stablecoin_df = stablecoin_df.merge(dai_df, on='date', how='outer')
stablecoin_df = stablecoin_df.sort_values('date').reset_index(drop=True)

# Calculate total stablecoin market cap
stablecoin_df['total_stablecoin_mcap'] = (
    stablecoin_df['usdc_mcap'].fillna(0) + 
    stablecoin_df['usdt_mcap'].fillna(0) + 
    stablecoin_df['dai_mcap'].fillna(0)
)

print(f"Shape: {stablecoin_df.shape}")
print(f"Date range: {stablecoin_df['date'].min()} to {stablecoin_df['date'].max()}")
print("\nFirst few rows:")
print(stablecoin_df.head())
print("\nLast few rows:")
print(stablecoin_df.tail())

Fetching USDC...
Fetching USDT...
Fetching DAI...
Shape: (2923, 5)
Date range: 2017-11-28 19:00:00 to 2025-11-28 19:00:00

First few rows:
                 date  usdc_mcap  usdt_mcap  dai_mcap  total_stablecoin_mcap
0 2017-11-28 19:00:00   109970.0        NaN       NaN               109970.0
1 2017-11-29 19:00:00   109970.0        NaN       NaN               109970.0
2 2017-11-30 19:00:00   109970.0        NaN       NaN               109970.0
3 2017-12-01 19:00:00   109970.0        NaN       NaN               109970.0
4 2017-12-02 19:00:00   109970.0        NaN       NaN               109970.0

Last few rows:
                    date     usdc_mcap     usdt_mcap      dai_mcap  \
2918 2025-11-24 19:00:00  1.845206e+11  7.431023e+10  4.844847e+08   
2919 2025-11-25 19:00:00  1.845072e+11  7.476404e+10  4.844847e+08   
2920 2025-11-26 19:00:00  1.845400e+11  7.550252e+10  4.844847e+08   
2921 2025-11-27 19:00:00  1.845403e+11  7.575451e+10  4.844847e+08   
2922 2025-11-28 19:00:00  1.84590

In [8]:
# --- Import FRED --- # 

fred = Fred(api_key='5064c9ff11c123b88035221b8faecff1')

# Get 10-Year Treasury yield
treasury_df = fred.get_series('DGS10', observation_start='2020-01-01').reset_index()
treasury_df.columns = ['date', 'treasury_yield']

# Get M2 Money Supply 
m2_df = fred.get_series('M2SL', observation_start='2020-01-01').reset_index()
m2_df.columns = ['date', 'm2_supply']

print(f"\nTreasury Data:")
print(f"Shape: {treasury_df.shape}")
print(f"Date range: {treasury_df['date'].min()} to {treasury_df['date'].max()}")
print("\nFirst few rows:")
print(treasury_df.head())
print("\nLast few rows:")
print(treasury_df.tail())

print(f"\nM2 Data:")
print(f"Shape: {m2_df.shape}")
print(f"Date range: {m2_df['date'].min()} to {m2_df['date'].max()}")
print("\nFirst few rows:")
print(m2_df.head())
print("\nLast few rows:")
print(m2_df.tail())


Treasury Data:
Shape: (1541, 2)
Date range: 2020-01-01 00:00:00 to 2025-11-26 00:00:00

First few rows:
        date  treasury_yield
0 2020-01-01             NaN
1 2020-01-02            1.88
2 2020-01-03            1.80
3 2020-01-06            1.81
4 2020-01-07            1.83

Last few rows:
           date  treasury_yield
1536 2025-11-20            4.10
1537 2025-11-21            4.06
1538 2025-11-24            4.04
1539 2025-11-25            4.01
1540 2025-11-26            4.00

M2 Data:
Shape: (70, 2)
Date range: 2020-01-01 00:00:00 to 2025-10-01 00:00:00

First few rows:
        date  m2_supply
0 2020-01-01    15416.3
1 2020-02-01    15466.6
2 2020-03-01    15993.4
3 2020-04-01    17005.4
4 2020-05-01    17852.9

Last few rows:
         date  m2_supply
65 2025-06-01    21942.4
66 2025-07-01    22028.8
67 2025-08-01    22108.4
68 2025-09-01    22212.5
69 2025-10-01    22298.1


In [9]:
final_df = rwa_df.copy()

# Ensure dates are datetime
final_df['date'] = pd.to_datetime(final_df['date']).dt.date
stablecoin_df['date'] = pd.to_datetime(stablecoin_df['date']).dt.date
treasury_df['date'] = pd.to_datetime(treasury_df['date']).dt.date
m2_df['date'] = pd.to_datetime(m2_df['date']).dt.date

# Merge stablecoin data
final_df = final_df.merge(stablecoin_df, on='date', how='left')

# Merge treasury data
final_df = final_df.merge(treasury_df, on='date', how='left')

print(f"\nFinal Merged Dataset:")
print(f"Shape: {final_df.shape}")
print(f"Columns: {final_df.columns.tolist()}")
print("\nFirst few rows:")
print(final_df.head())
print("\nLast few rows:")
print(final_df.tail())


Final Merged Dataset:
Shape: (59973, 9)
Columns: ['date', 'protocol', 'tvl', 'chain', 'usdc_mcap', 'usdt_mcap', 'dai_mcap', 'total_stablecoin_mcap', 'treasury_yield']

First few rows:
         date      protocol  tvl chain     usdc_mcap     usdt_mcap  \
0  2023-01-26  ondo-finance  135   NaN  6.731793e+10  4.288316e+10   
1  2023-01-27  ondo-finance  315   NaN  6.749709e+10  4.265105e+10   
2  2023-01-28  ondo-finance  315   NaN  6.761286e+10  4.268658e+10   
3  2023-01-29  ondo-finance  315   NaN  6.776868e+10  4.256662e+10   
4  2023-01-30  ondo-finance  495   NaN  6.780398e+10  4.215550e+10   

      dai_mcap  total_stablecoin_mcap  treasury_yield  
0  943742315.0           1.111448e+11            3.49  
1  944479138.0           1.110926e+11            3.52  
2  944375897.0           1.112438e+11             NaN  
3  944369497.0           1.112797e+11             NaN  
4  945589427.0           1.109051e+11            3.55  

Last few rows:
             date       protocol   tvl    

#### Data Cleaning

In [10]:
# --- Sanity Check: NaNs --- #

print("Dataset overview:")
print(f"Shape: {final_df.shape}")
print(f"Columns: {final_df.columns.tolist()}")

# Count nulls per column
null_counts = final_df.isnull().sum()
null_pct = (final_df.isnull().sum() / len(final_df) * 100).round(2)

# Create summary table
null_summary = pd.DataFrame({
    'column': null_counts.index,
    'null_count': null_counts.values,
    'null_percentage': null_pct.values
}).sort_values('null_count', ascending=False)

print("\nNull values by column:")
print(null_summary)

print("\nColumns with nulls (>0%):")
print(null_summary[null_summary['null_count'] > 0])

print("\nChain column null count:")
print(f"  Nulls: {final_df['chain'].isnull().sum()} ({final_df['chain'].isnull().sum()/len(final_df)*100:.2f}%)")

print("\nTreasury yield null count:")
print(f"  Nulls: {final_df['treasury_yield'].isnull().sum()} ({final_df['treasury_yield'].isnull().sum()/len(final_df)*100:.2f}%)")

Dataset overview:
Shape: (59973, 9)
Columns: ['date', 'protocol', 'tvl', 'chain', 'usdc_mcap', 'usdt_mcap', 'dai_mcap', 'total_stablecoin_mcap', 'treasury_yield']

Null values by column:
                  column  null_count  null_percentage
8         treasury_yield       19045            31.76
3                  chain       13046            21.75
4              usdc_mcap          53             0.09
5              usdt_mcap          53             0.09
6               dai_mcap          53             0.09
7  total_stablecoin_mcap          53             0.09
0                   date           0             0.00
1               protocol           0             0.00
2                    tvl           0             0.00

Columns with nulls (>0%):
                  column  null_count  null_percentage
8         treasury_yield       19045            31.76
3                  chain       13046            21.75
4              usdc_mcap          53             0.09
5              usdt_mcap      

In [12]:
# Sort by date 
final_df = final_df.sort_values('date').reset_index(drop=True)

# Fill chain column  
final_df['chain'] = final_df['chain'].fillna('All')

# Forward fill macros column 
final_df['treasury_yield'] = final_df['treasury_yield'].fillna(method='ffill')

# Backwards fill 
final_df['treasury_yield'] = final_df['treasury_yield'].fillna(method='bfill')

# Forward fill stablecoin data
final_df['usdc_mcap'] = final_df['usdc_mcap'].fillna(method='ffill')
final_df['usdt_mcap'] = final_df['usdt_mcap'].fillna(method='ffill')
final_df['dai_mcap'] = final_df['dai_mcap'].fillna(method='ffill')
final_df['total_stablecoin_mcap'] = final_df['total_stablecoin_mcap'].fillna(method='ffill')

# Backward fill stablecoins too
final_df['usdc_mcap'] = final_df['usdc_mcap'].fillna(method='bfill')
final_df['usdt_mcap'] = final_df['usdt_mcap'].fillna(method='bfill')
final_df['dai_mcap'] = final_df['dai_mcap'].fillna(method='bfill')
final_df['total_stablecoin_mcap'] = final_df['total_stablecoin_mcap'].fillna(method='bfill')

# Recalculate tvl_share_by_chain after filling chain
chain_totals = final_df.groupby(['date', 'chain'])['tvl'].transform('sum')
final_df['tvl_share_by_chain'] = (final_df['tvl'] / chain_totals * 100).fillna(0)

/var/folders/tz/kzg31v8n6zz4brsskxfdbc0c0000gn/T/ipykernel_82037/4076839867.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  final_df['treasury_yield'] = final_df['treasury_yield'].fillna(method='ffill')
/var/folders/tz/kzg31v8n6zz4brsskxfdbc0c0000gn/T/ipykernel_82037/4076839867.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  final_df['treasury_yield'] = final_df['treasury_yield'].fillna(method='bfill')
/var/folders/tz/kzg31v8n6zz4brsskxfdbc0c0000gn/T/ipykernel_82037/4076839867.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  final_df['usdc_mcap'] = final_df['usdc_mcap'].fillna(method='ffill')
/var/folders/tz/kzg31v8n6zz4brsskxfdbc0c0000gn/T/ipykernel_82037/4076839867.py:15: FutureWarning: Series.fillna with 'me

In [13]:
# --- Sanity check: NaNs cleaned? --- #
print(f"\nCleaned shape: {final_df.shape}")
print(f"Date range: {final_df['date'].min()} to {final_df['date'].max()}")

print("\nFirst few rows:")
print(final_df.head())

print("\nLast few rows:")
print(final_df.tail())


Cleaned shape: (59973, 10)
Date range: 2020-10-17 to 2025-11-29

First few rows:
         date    protocol    tvl              chain     usdc_mcap  \
0  2020-10-17  centrifuge  20165           Ethereum  1.472977e+10   
1  2020-10-17  centrifuge      0  Ethereum-borrowed  1.472977e+10   
2  2020-10-17  centrifuge  20165                All  1.472977e+10   
3  2020-10-17  centrifuge      0           borrowed  1.472977e+10   
4  2020-10-18  centrifuge      0  Ethereum-borrowed  1.472977e+10   

      usdt_mcap     dai_mcap  total_stablecoin_mcap  treasury_yield  \
0  2.711217e+09  338938397.0           1.777992e+10            0.78   
1  2.711217e+09  338938397.0           1.777992e+10            0.78   
2  2.711217e+09  338938397.0           1.777992e+10            0.78   
3  2.711217e+09  338938397.0           1.777992e+10            0.78   
4  2.733321e+09  355118575.0           1.781821e+10            0.78   

   tvl_share_by_chain  
0               100.0  
1                 0.0  
2   

In [14]:
# --- Add ASSET_TYPE categorical variable --- #

def categorize_protocol(protocol):
    """Categorize protocols by primary asset type"""
    treasury_protocols = ['ondo-finance', 'backed-finance']
    credit_protocols = ['maple-finance', 'goldfinch', 'truefi', 'credix', 'ribbon-lend']
    diversified_protocols = ['centrifuge', 'polytrade', 'swarm-markets']
    
    if protocol in treasury_protocols:
        return 'Treasury'
    elif protocol in credit_protocols:
        return 'Private Credit'
    elif protocol in diversified_protocols:
        return 'Diversified'
    else:
        return 'Other'

# Add asset_type column
final_df['asset_type'] = final_df['protocol'].apply(categorize_protocol)

# Check distribution
print("\nAsset type distribution:")
print(final_df['asset_type'].value_counts())


Asset type distribution:
asset_type
Private Credit    37943
Diversified       15155
Treasury           6875
Name: count, dtype: int64


In [15]:
# --- Final sanity checks --- # 

# Verify no nulls remain
print("\nFinal null check:")
print(final_df.isnull().sum())

# Summary stats
print(f"\nFinal dataset summary:")
print(f"Shape: {final_df.shape}")
print(f"Columns: {final_df.columns.tolist()}")
print(f"Date range: {final_df['date'].min()} to {final_df['date'].max()}")
print(f"Number of protocols: {final_df['protocol'].nunique()}")
print(f"Number of chains: {final_df['chain'].nunique()}")


Final null check:
date                     0
protocol                 0
tvl                      0
chain                    0
usdc_mcap                0
usdt_mcap                0
dai_mcap                 0
total_stablecoin_mcap    0
treasury_yield           0
tvl_share_by_chain       0
asset_type               0
dtype: int64

Final dataset summary:
Shape: (59973, 11)
Columns: ['date', 'protocol', 'tvl', 'chain', 'usdc_mcap', 'usdt_mcap', 'dai_mcap', 'total_stablecoin_mcap', 'treasury_yield', 'tvl_share_by_chain', 'asset_type']
Date range: 2020-10-17 to 2025-11-29
Number of protocols: 9
Number of chains: 21


In [16]:
# -- Create M2 liqudity gauge data -- # 

# 1. Create M2 series 
m2_series = (
    m2_df
    .drop_duplicates(subset='date')
    .sort_values('date')
    .copy() 
)

print(f"M2 monthly observations: {len(m2_series)}")
print(f"Date range: {m2_series['date'].min()} to {m2_series['date'].max()}")

# 2. Calculate MoM M2 growth rate 
m2_series['m2_growth'] = m2_series['m2_supply'].pct_change() * 100 

# 3. Group into liquidity categories 
def categorize_m2_environment(growth_rate):
    """ Group calculated m2 growth into liquidity categories """
    if pd.isna(growth_rate):
        return 'Neutral'
    elif growth_rate > 2.0:
        return 'Expansion'      # strong liquidity expansion
    elif growth_rate > 0.5:
        return 'Growth'         # moderate growth
    elif growth_rate > -0.5:
        return 'Neutral'        # stable
    elif growth_rate > -2.0:
        return 'Contraction'    # tightening
    else:
        return 'Crisis'         # warning! severe liquidity contraction 

# 4. Apply categorization to series 
m2_series['liquidity_environment'] = m2_series['m2_growth'].apply(categorize_m2_environment)

# 5. Sanity check 
env_dist = m2_series['liquidity_environment'].value_counts()
print(env_dist)
print(f"\nPercentages:")
print((env_dist / len(m2_series) * 100).round(1))

print("\nM2 Growth Rate Statistics")
print(m2_series['m2_growth'].describe())

# 6. Save as csv 
m2_output_path = '/Users/erikasohn/Library/Mobile Documents/com~apple~CloudDocs/Documents/classes/khoury/ds4200_fa25/coin-project/data/processed/m2_liquidity_gauge.csv'
m2_series.to_csv(m2_output_path, index=False)
print(f"\nM2 gauge data saved to: {m2_output_path}")

M2 monthly observations: 70
Date range: 2020-01-01 to 2025-10-01
liquidity_environment
Neutral        44
Growth         20
Expansion       3
Contraction     3
Name: count, dtype: int64

Percentages:
liquidity_environment
Neutral        62.9
Growth         28.6
Expansion       4.3
Contraction     4.3
Name: count, dtype: float64

M2 Growth Rate Statistics
count    69.000000
mean      0.542265
std       1.113155
min      -1.456253
25%       0.022226
50%       0.343949
75%       0.776964
max       6.327610
Name: m2_growth, dtype: float64

M2 gauge data saved to: /Users/erikasohn/Library/Mobile Documents/com~apple~CloudDocs/Documents/classes/khoury/ds4200_fa25/coin-project/data/processed/m2_liquidity_gauge.csv


In [18]:
print(final_df.columns.tolist())

['date', 'protocol', 'tvl', 'chain', 'usdc_mcap', 'usdt_mcap', 'dai_mcap', 'total_stablecoin_mcap', 'treasury_yield', 'tvl_share_by_chain', 'asset_type', 'tvl_millions', 'tvl_pct_of_stablecoin', 'year', 'month', 'quarter', 'tvl_growth_rate', 'yield_regime', 'usdc_composition', 'usdt_composition']


In [17]:
# -- Feature engineering -- # 

# 1. TVL metrics 
final_df['tvl_millions'] = final_df['tvl'] / 1_000_000
final_df['tvl_pct_of_stablecoin'] = (final_df['tvl'] / final_df['total_stablecoin_mcap']) * 100

# 2. Time-series features 
final_df['year'] = pd.to_datetime(final_df['date']).dt.year
final_df['month'] = pd.to_datetime(final_df['date']).dt.month
final_df['quarter'] = pd.to_datetime(final_df['date']).dt.quarter

# 3. Growth metrics 
final_df = final_df.sort_values(['protocol', 'date'])
final_df['tvl_growth_rate'] = final_df.groupby('protocol')['tvl'].pct_change() * 100

# 4. Treasury yield categories 
def categorize_yield(yield_val):
    if yield_val < 2.0:
        return 'Low'
    elif yield_val < 4.0:
        return 'Moderate'
    else:
        return 'High'

final_df['yield_regime'] = final_df['treasury_yield'].apply(categorize_yield)

# 5. Stablecoin composition (USDC vs USDT)
final_df['usdc_composition'] = (final_df['usdc_mcap'] / final_df['total_stablecoin_mcap']) * 100
final_df['usdt_composition'] = (final_df['usdt_mcap'] / final_df['total_stablecoin_mcap']) * 100

print(f"Total features: {final_df.shape[1]}")

Total features: 20


In [19]:
# --- Save cleaned data --- #
final_df.to_csv('/Users/erikasohn/Library/Mobile Documents/com~apple~CloudDocs/Documents/classes/khoury/ds4200_fa25/coin-project/data/processed/final_rwa_data.csv', index=False)